In [1]:
import pickle, gzip
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob, os
import re, ast
import itertools
import seaborn as sns
from matplotlib import rcParams
from qiskit.visualization import plot_histogram
from matplotlib import colors
from matplotlib.colors import LinearSegmentedColormap
from seaborn import set_theme
from matplotlib import style

rcParams.update({'figure.autolayout': True})
rcParams['pdf.fonttype'] = 42
rcParams['ps.fonttype'] = 42
#rc('text', usetex=True)
rcParams['text.usetex'] = True

In [2]:

####### original version
# # Compute the new QVF for the whole circuit, as well as for each available qubit
def compute_QVF_michelson_contrast_single_injection(df, circuit_name, phi, theta):
    dfFilter = df[(df.circuit_name==circuit_name) & (df.first_phi==phi) & (df.first_theta==theta)]
    QVF = {}
    QVF['QVF_circuit'] = dfFilter['QVF'].mean()
    
    qubits = set(dfFilter['first_qubit_injected'])    
    for q in qubits:
        QVF['QVF_qubit_'+str(q)] = dfFilter[dfFilter.first_qubit_injected==q]['QVF'].mean()
    return QVF
    

#######multi-injection version
# Compute the new QVF for the whole circuit, as well as for each available qubit
def compute_QVF_michelson_contrast_double_injection(df, circuit_name, phi_0, theta_0, phi_1, theta_1):
    dfFilter = df[(df.circuit_name == circuit_name) & (df.first_phi == phi_0) & (df.first_theta == theta_0)& (df.second_phi == phi_1) & (df.second_theta == theta_1)]
    QVF = {}
    QVF['QVF_circuit'] = dfFilter['QVF'].mean()


    qubits = set(dfFilter['first_qubit_injected'])
    for q in qubits:
        QVF['QVF_qubit_' +
            str(q)] = dfFilter[dfFilter.first_qubit_injected == q]['QVF'].mean()
    return QVF



def QVF_michelson_contrast(gold_bitstring, answer, shots):    
    # Sort the answer, position 0 has the highest bitstring, position 1 the second highest
    answer_sorted = sorted(answer, key=answer.get, reverse=True)
    
    # If gold bitstring is not in answer, percentage is zero
    if gold_bitstring not in answer:
        good_percent = 0
    else:
        good_percent = answer[gold_bitstring]/shots
        
    if answer_sorted[0] == gold_bitstring: # gold bitstring has the highest count (max)
        # next bitstring is the second highest
        next_percent = answer[answer_sorted[1]]/shots 
        next_bitstring = answer_sorted[1]
    else: # gold bitstring has NOT the highest count (not max)
        next_percent = answer[answer_sorted[0]]/shots 
        next_bitstring = answer_sorted[0]
    qvf = (good_percent - next_percent) / (good_percent + next_percent)    
    return 1 - (qvf+1)/2, next_bitstring
    
# Read pickled data and store results in a dataframe
def build_DF_newQVF(data):
    results = []
    shots = 1024
    gold_bitstring = max(data['output_gold_noise'], key=data['output_gold_noise'].get)#check
    original_gold_percentage = data['output_gold_noise'][gold_bitstring]/shots

    for i, answer in enumerate(data['output_injections_noise']):
        qvf, next_bitstring = QVF_michelson_contrast(gold_bitstring, answer, shots)
        max_key = max(answer, key=answer.get)
        output_percentage = answer[max_key]/shots
        next_bitstring_percentage = answer[next_bitstring]/shots
        if gold_bitstring not in answer:
            gold_percentage = 0
        else:
            gold_percentage = answer[gold_bitstring]/shots
        result = {'gold_bitstring':gold_bitstring
                , 'gold_count_percentage':gold_percentage
                , 'original_gold_count_percentage':original_gold_percentage
                , 'next_bitstring': next_bitstring
                , 'next_bitstring_percentage': next_bitstring_percentage
                , 'QVF':qvf
                , 'first_qubit_injected':data['circuits_injections'][i].metadata['first_qubit']                
                , 'first_phi':data['circuits_injections'][i].metadata['first_phi']
                , 'first_theta':data['circuits_injections'][i].metadata['first_theta']
                , 'second_qubit_injected':data['circuits_injections'][i].metadata['second_qubit']                
                , 'second_phi':data['circuits_injections'][i].metadata['second_phi']
                , 'second_theta':data['circuits_injections'][i].metadata['second_theta']
                , 'circuit_name':data['name']
                }
        #first_qubit, first_theta, first_phi, second_qubit, second_theta, second_phi
        results.append(result)
    return pd.DataFrame(results)


In [3]:
files_to_process = [
'../results/double_fi_BV_4qubits.p.gz', 
]
# read all data and insert it into one dataframe
df_newQVF = pd.DataFrame()
for filename in files_to_process:
    data = pickle.load(gzip.open(filename, 'r'))
    
    for d in data:
        df_newQVF = pd.concat([df_newQVF, build_DF_newQVF(d)], ignore_index=True)
    del data

In [4]:
df_only_single = df_newQVF[(df_newQVF.second_phi == 0) & (df_newQVF.second_theta == 0)]

In [5]:
# Get available circuits and parameters used for injection (phi and theta)
phi_list = list(set(df_newQVF.first_phi))
phi_list.sort(reverse=False)
theta_list = list(set(df_newQVF.first_theta))
theta_list.sort()
circuits = list(set(df_newQVF.circuit_name))
circuits.sort()

In [6]:
results = []
for circuit in circuits:
    for phi in phi_list:
        for theta in theta_list:
            qvf = compute_QVF_michelson_contrast_single_injection(df_newQVF, circuit, phi, theta)
            qvf['circuit_name'] = circuit
            qvf['first_phi'] = phi
            qvf['first_theta'] = theta
            #qvf['threshold'] = threshold
            results.append(qvf)

new_qvfDF_noise = pd.DataFrame(results)
new_qvfDF_noise = new_qvfDF_noise[sorted(list(new_qvfDF_noise.columns))]
new_qvfDF_noise

,QVF_circuit,QVF_qubit_0,QVF_qubit_1,QVF_qubit_2,QVF_qubit_3,circuit_name,first_phi,first_theta
0,0.078482,0.079472,0.078145,0.072333,0.082848,Bernstein-Vazirani_4,0.000000,0.000000
1,0.089887,0.103355,0.077680,0.081494,0.095708,Bernstein-Vazirani_4,0.000000,0.261799
2,0.126586,0.127177,0.109425,0.141111,0.121906,Bernstein-Vazirani_4,0.000000,0.523599
3,0.189041,0.200108,0.176740,0.205174,0.177404,Bernstein-Vazirani_4,0.000000,0.785398
4,0.275555,0.283009,0.269219,0.299167,0.257427,Bernstein-Vazirani_4,0.000000,1.047198
...,...,...,...,...,...,...,...,...
164,0.524662,0.599008,0.565924,0.553399,0.459756,Bernstein-Vazirani_4,3.141593,2.094395
165,0.534153,0.619674,0.589208,0.564287,0.458837,Bernstein-Vazirani_4,3.141593,2.356194
166,0.549634,0.641532,0.619342,0.570987,0.473018,Bernstein-Vazirani_4,3.141593,2.617994
167,0.564919,0.654824,0.644796,0.576138,0.492836,Bernstein-Vazirani_4,3.141593,2.879793


In [7]:
results = []
for circuit in circuits:
    for phi in phi_list:
        for theta in theta_list:
            qvf = compute_QVF_michelson_contrast_single_injection(df_only_single, circuit, phi, theta)
            qvf['circuit_name'] = circuit
            qvf['first_phi'] = phi
            qvf['first_theta'] = theta
            #qvf['threshold'] = threshold
            results.append(qvf)

new_qvfDF_noise_single = pd.DataFrame(results)
new_qvfDF_noise_single = new_qvfDF_noise_single[sorted(list(new_qvfDF_noise_single.columns))]
new_qvfDF_noise_single

,QVF_circuit,QVF_qubit_0,QVF_qubit_1,QVF_qubit_2,QVF_qubit_3,circuit_name,first_phi,first_theta
0,0.078482,0.079472,0.078145,0.072333,0.082848,Bernstein-Vazirani_4,0.000000,0.000000
1,0.085274,0.098567,0.068130,0.080519,0.090286,Bernstein-Vazirani_4,0.000000,0.261799
2,0.120796,0.122758,0.102016,0.139223,0.113283,Bernstein-Vazirani_4,0.000000,0.523599
3,0.180173,0.185054,0.175937,0.213571,0.154882,Bernstein-Vazirani_4,0.000000,0.785398
4,0.259663,0.277020,0.252859,0.308793,0.218858,Bernstein-Vazirani_4,0.000000,1.047198
...,...,...,...,...,...,...,...,...
164,0.349586,0.440014,0.400160,0.437958,0.230431,Bernstein-Vazirani_4,3.141593,2.094395
165,0.292490,0.388105,0.362115,0.403787,0.147052,Bernstein-Vazirani_4,3.141593,2.356194
166,0.268309,0.385029,0.364954,0.384995,0.100784,Bernstein-Vazirani_4,3.141593,2.617994
167,0.265519,0.371213,0.381595,0.370523,0.103602,Bernstein-Vazirani_4,3.141593,2.879793


In [8]:
# comparing distribution of single FI vs double FI
dirHistPlots = '../plots/double_FI/histograms/'
if not os.path.exists(dirHistPlots):
    os.makedirs(dirHistPlots) 


circs = [new_qvfDF_noise_single, new_qvfDF_noise]
qvf_tmp = list()
for i, dfCirc in enumerate(circs):
    qvf_tmp.append(dfCirc)
    qvf_tmp[i] = qvf_tmp[i].pivot('first_phi', 'first_theta', 'QVF_circuit')
    qvf_tmp[i].columns.name = '$\\theta$ shift'
    qvf_tmp[i].index.name = '$\\phi$ shift'

    all_values = []
    for column in qvf_tmp[i]:
        this_column_values = qvf_tmp[i][column].tolist()
        all_values += this_column_values
    one_column_df = pd.DataFrame(all_values)
    print('mean:',one_column_df.mean()[0],' std:',one_column_df.std()[0])
print()

fig, ax = plt.subplots(1, 1, figsize=(6, 5))
sns.set(font_scale=1.3)
#ax = 
colorsDist = ['black', 'red', 'green', 'blue']
for i, data in enumerate(qvf_tmp):
    sns.distplot(data, bins=256, color=colorsDist[i])
plt.xlim(0, 1)

tmpFileName = dirHistPlots+'BV_merged_distribution_histogram'+'.pdf'
fig.savefig(tmpFileName, bbox_inches = 'tight')
plt.close()

mean: 0.4646965431469038  std: 0.18182855069419476
mean: 0.5338265410657026  std: 0.1839001395962708



/usr/local/lib/python3.10/site-packages/seaborn/distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.10/site-packages/seaborn/distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


In [9]:
# Heatmaps for single and double FI
##considering only phi angles up to pi
theta_list_tex = ['0', '', '', '$\\frac{\pi}{4}$', '', '', '$\\frac{\pi}{2}$', ''
                , '', '$\\frac{3\pi}{4}$', '', '', '$\pi$']
phi_list_tex = theta_list_tex

dirHeatmapPlots = '../plots/double_FI/heatmaps/'
if not os.path.exists(dirHeatmapPlots):
    os.makedirs(dirHeatmapPlots)

circs = [new_qvfDF_noise_single, new_qvfDF_noise]            
for i,circuitDF in enumerate(circs):
    qvf_tmp = circuitDF
    qvf_tmp = qvf_tmp.pivot('first_phi', 'first_theta', 'QVF_circuit')
    qvf_tmp.columns.name = '$\\theta$ shift'
    qvf_tmp.index.name = '$\\phi$ shift'
    fig, ax = plt.subplots(1, 1, figsize=(6, 5))
    param={'label': 'QVF'}

    divnorm = colors.TwoSlopeNorm(vmin=0, vcenter=0.5, vmax=1)
    rdgn = sns.diverging_palette(h_neg=130, h_pos=10, s=200, l=55, sep=20, as_cmap=True)
    sns.set(font_scale=1.3)
    ax = sns.heatmap(qvf_tmp, xticklabels=theta_list_tex, yticklabels=phi_list_tex, cmap=rdgn, cbar_kws=param, vmin=0, vmax=1)
    if i == 0:
        fig.savefig(dirHeatmapPlots+'BV_single_heatmap.pdf', bbox_inches='tight')
    else:
        fig.savefig(dirHeatmapPlots+'BV_double_heatmap.pdf', bbox_inches='tight')
    plt.close()
    
    
###########
# Plot delta heatmaps between single and double FI
dirPlots = '../plots/double_FI/deltaHeatmaps/'
if not os.path.exists(dirPlots):
    os.makedirs(dirPlots)

        
qvf_tmp = new_qvfDF_noise_single.copy()
qvf_tmp['delta'] = new_qvfDF_noise['QVF_circuit'] - new_qvfDF_noise_single['QVF_circuit']
qvf_tmp = qvf_tmp.pivot('first_phi', 'first_theta', 'delta')
qvf_tmp.columns.name = '$\\theta$ shift'
qvf_tmp.index.name = '$\\phi$ shift'
fig, ax = plt.subplots(1, 1, figsize=(6, 5))
label = '$\Delta$QVF = Double - Single'
param={'label': label}
ax = sns.heatmap(qvf_tmp, xticklabels=theta_list_tex, yticklabels=phi_list_tex, cmap='seismic', cbar_kws=param, vmin=-1, vmax=1)            
fig.savefig(dirPlots+'BV_double_single_delta_heatmap.pdf', bbox_inches='tight')
plt.close()


In [10]:
##considering only phi angles up to pi
theta_list_tex = ['0', '', '', '$\\frac{\pi}{4}$', '', '', '$\\frac{\pi}{2}$', ''
                , '', '$\\frac{3\pi}{4}$', '', '', '$\pi$']
phi_list_tex = ['$\pi$', '', '',  '$\\frac{3\pi}{4}$'  
            , '', '', '$\\frac{\pi}{2}$', '', '', '$\\frac{\pi}{4}$'
            , '', '', '0']



dirHeatmapPlots = '../plots/double_FI/heatmaps/'
dirHistPlots = '../plots/double_FI/histograms/'
if not os.path.exists(dirHeatmapPlots):
    os.makedirs(dirHeatmapPlots)
if not os.path.exists(dirHistPlots):
    os.makedirs(dirHistPlots)    
            
for circuit in circuits:
    # get a list of qubit columns (circuit may have different number of qubits now)
    colNames = new_qvfDF_noise[new_qvfDF_noise['circuit_name']==circuit].dropna(axis=1).columns    
    QVF_list= ['QVF_circuit']
    QVF_list.extend( [x for x in colNames if re.search('QVF_qubit_.*',x)] ) # uncomment this line to include individual qubit analysis
    
    for qvf_idx in QVF_list:
        qvf_tmp = new_qvfDF_noise[new_qvfDF_noise['circuit_name']==circuit]
        qvf_tmp = qvf_tmp.pivot('first_phi', 'first_theta', qvf_idx)
        qvf_tmp.columns.name = '$\\theta$ shift'
        qvf_tmp.index.name = '$\\phi$ shift'
        fig, ax = plt.subplots(1, 1, figsize=(6, 5))
        param={'label': 'QVF'}

        divnorm = colors.TwoSlopeNorm(vmin=0, vcenter=0.5, vmax=1)
        rdgn = sns.diverging_palette(h_neg=130, h_pos=10, s=200, l=55, sep=20, as_cmap=True)
        sns.set(font_scale=1.3)
        ax = sns.heatmap(qvf_tmp, xticklabels=theta_list_tex, yticklabels=phi_list_tex, cmap=rdgn, cbar_kws=param, vmin=0, vmax=1)
        fig.savefig(dirHeatmapPlots+circuit+'_'+qvf_idx+'_heatmap.pdf', bbox_inches='tight')
        plt.close()

        
        all_values = []
        for column in qvf_tmp:
            this_column_values = qvf_tmp[column].tolist()
            all_values += this_column_values
        one_column_df = pd.DataFrame(all_values)

        fig, ax = plt.subplots(1, 1, figsize=(6, 5))
        sns.set(font_scale=1.3)
        ax = sns.distplot(qvf_tmp, bins=256, color='black')
        plt.xlim(0, 1)

        tmp_mean = one_column_df.mean()
        tmp_stddev = one_column_df.std()
        ax.get_yaxis().set_visible(False)
        tmpFileName = dirHistPlots+circuit+'_'+qvf_idx+'_distribution_histogram_'+str(tmp_mean[0])+'_'+str(tmp_stddev[0])+'.pdf'
        fig.savefig(tmpFileName, bbox_inches = 'tight')
        plt.close()

/usr/local/lib/python3.10/site-packages/seaborn/distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.10/site-packages/seaborn/distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.10/site-packages/seaborn/distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level functi

In [11]:
# Plot delta heatmaps
dirPlots = '../plots/double_FI/deltaHeatmaps/'
if not os.path.exists(dirPlots):
    os.makedirs(dirPlots)

for circuit in circuits:
    # get a list of qubit columns (circuit may have different number of qubits now)
    colNames = new_qvfDF_noise[new_qvfDF_noise['circuit_name']==circuit].dropna(axis=1).columns    
    QVF_list =  [x for x in colNames if re.search('QVF_qubit_.*',x)]  # uncomment this line to include individual qubit analysis    
    
    for pair in itertools.combinations(QVF_list, 2):
            qvf_tmp = new_qvfDF_noise[new_qvfDF_noise['circuit_name']==circuit].copy()
            qvf_tmp['delta'] = qvf_tmp[pair[0]] - qvf_tmp[pair[1]]        
            qvf_tmp = qvf_tmp.pivot('first_phi', 'first_theta', 'delta')
            qvf_tmp.columns.name = '$\\theta$ shift'
            qvf_tmp.index.name = '$\\phi$ shift'
            fig, ax = plt.subplots(1, 1, figsize=(5, 6))
            label = '$\Delta$QVF = '+pair[0].replace('_','')+' - '+pair[1].replace('_','')
            param={'label': label}
            ax = sns.heatmap(qvf_tmp, xticklabels=theta_list_tex, yticklabels=phi_list_tex, cmap='seismic', cbar_kws=param, vmin=-1, vmax=1)            
            plt.axhline(y=9.5, color="blue", linestyle="--")       #T at phi=pi/4
            plt.text(13.25, 9.7, r'$T$', fontsize=10, color="blue")            
            plt.axhline(y=6.5, color="black", linestyle="--")      #S at phi=pi/2  
            plt.text(13.25, 6.7, r'$S$', fontsize=10, color="black")       
            plt.axhline(y=0.5, color="cyan", linestyle="--")       #Z at phi=pi   
            plt.text(13.25, 0.7, r'$Z$', fontsize=10, color="cyan")          
            plt.axvline(x=12.5, color="purple", linestyle="--")     #X,Y at theta=pi
            plt.text(12, -0.5, r'$X, Y$', fontsize=10, color="purple")
            fig.savefig(dirPlots+circuit+'_'+pair[0]+'-'+pair[1]+'_delta_heatmap.pdf', bbox_inches='tight')
            plt.close()


In [8]:
# Get available circuits and parameters used for injection (phi and theta)
phi_list = list(set(df_newQVF.first_phi))
phi_list.sort(reverse=True)
theta_list = list(set(df_newQVF.first_theta))
theta_list.sort()
circuits = list(set(df_newQVF.circuit_name))
circuits.sort()


In [9]:
results = []
for circuit in circuits:
    for phi_0 in phi_list:
        for theta_0 in theta_list:
            for phi_1 in [phi_1 for phi_1 in phi_list if phi_1<=phi_0]:
                for theta_1 in [theta_1 for theta_1 in theta_list if theta_1<=theta_0]:
                    qvf = compute_QVF_michelson_contrast_double_injection(
                        df_newQVF, circuit, phi_0, theta_0, phi_1, theta_1)
                    qvf['circuitName'] = circuit
                    qvf['firstPhi'] = phi_0
                    qvf['firstTheta'] = theta_0
                    qvf['secondPhi'] = phi_1
                    qvf['secondTheta'] = theta_1
                    results.append(qvf)

new_qvfDF_noise = pd.DataFrame(results)
new_qvfDF_noise = new_qvfDF_noise[sorted(list(new_qvfDF_noise.columns))]
new_qvfDF_noise

,QVF_circuit,QVF_qubit_0,QVF_qubit_1,QVF_qubit_2,QVF_qubit_3,circuitName,firstPhi,firstTheta,secondPhi,secondTheta
0,0.797816,0.679727,0.954295,0.681061,0.870987,Bernstein-Vazirani_4,3.141593,0.000000,3.141593,0.000000
1,0.790884,0.668484,0.951829,0.674911,0.863411,Bernstein-Vazirani_4,3.141593,0.000000,2.879793,0.000000
2,0.786730,0.676650,0.930405,0.672787,0.859588,Bernstein-Vazirani_4,3.141593,0.000000,2.617994,0.000000
3,0.776393,0.662405,0.917520,0.669996,0.846014,Bernstein-Vazirani_4,3.141593,0.000000,2.356194,0.000000
4,0.760491,0.665517,0.887330,0.653948,0.828450,Bernstein-Vazirani_4,3.141593,0.000000,2.094395,0.000000
...,...,...,...,...,...,...,...,...,...,...
8276,0.901019,0.950205,0.972976,0.959164,0.811982,Bernstein-Vazirani_4,0.000000,3.141593,0.000000,2.094395
8277,0.914017,0.966858,0.984873,0.972880,0.823485,Bernstein-Vazirani_4,0.000000,3.141593,0.000000,2.356194
8278,0.918240,0.959686,0.993889,0.971676,0.834253,Bernstein-Vazirani_4,0.000000,3.141593,0.000000,2.617994
8279,0.915945,0.950786,0.991230,0.974499,0.830732,Bernstein-Vazirani_4,0.000000,3.141593,0.000000,2.879793


In [ ]:
##considering only phi angles up to pi
theta_list_tex = ['0', '', '', '$\\frac{\pi}{4}$', '', '',
                  '$\\frac{\pi}{2}$', '', '', '$\\frac{3\pi}{4}$', '', '', '$\pi$']
phi_list_tex = ['0', '', '', '$\\frac{\pi}{4}$', '', '',
                  '$\\frac{\pi}{2}$', '', '', '$\\frac{3\pi}{4}$', '', '', '$\pi$']


dirHeatmapPlots = '../plots/double_FI/heatmaps/'
dirHistPlots = '../plots/double_FI/histograms/'

if not os.path.exists(dirHeatmapPlots):
    os.makedirs(dirHeatmapPlots)
if not os.path.exists(dirHistPlots):
    os.makedirs(dirHistPlots)



for circuit in circuits:
    # get a list of qubit columns (circuit may have different number of qubits now)
    colNames = new_qvfDF_noise[new_qvfDF_noise['circuitName'] == circuit].dropna(
        axis=1).columns
    QVF_list = ['QVF_circuit']
    # uncomment this line to include individual qubit analysis
    QVF_list.extend([x for x in colNames if re.search('QVF_qubit_.*', x)])

    for qvf_idx in QVF_list:
        for phi_0 in phi_list:
            for theta_0 in theta_list:
                qvf_tmp = new_qvfDF_noise[(new_qvfDF_noise['circuitName'] == circuit) & (new_qvfDF_noise['firstPhi'] == phi_0) & (new_qvfDF_noise['firstTheta'] == theta_0)]
                qvf_tmp = qvf_tmp.pivot('secondPhi', 'secondTheta', qvf_idx)
                qvf_tmp.columns.name = '$\\theta$ shift'
                qvf_tmp.index.name = '$\\phi$ shift'
                fig, ax = plt.subplots(1, 1, figsize=(6, 5))
                param = {'label': 'QVF'}

                divnorm = colors.TwoSlopeNorm(vmin=0, vcenter=0.5, vmax=1)
                rdgn = sns.diverging_palette(
                    h_neg=130, h_pos=10, s=200, l=55, sep=20, as_cmap=True)
                sns.set(font_scale=1.3)
                ax = sns.heatmap(qvf_tmp, xticklabels=theta_list_tex, yticklabels=phi_list_tex, cmap=rdgn, cbar_kws=param, vmin=0, vmax=1)

                ax.invert_yaxis()

                fig.savefig(dirHeatmapPlots+circuit+'_th_0_'+str(theta_0)+'_ph_0_'+str(phi_0)+'_'+qvf_idx +'_heatmap.pdf', bbox_inches='tight')
                plt.close()

                all_values = []
                for column in qvf_tmp:
                    this_column_values = qvf_tmp[column].tolist()
                    all_values += this_column_values
                one_column_df = pd.DataFrame(all_values)

                fig, ax = plt.subplots(1, 1, figsize=(6, 5))
                sns.set(font_scale=1.3)
                ax = sns.distplot(qvf_tmp, bins=256, color='black')
                plt.xlim(0, 1)



                tmp_mean = one_column_df.mean()
                tmp_stddev = one_column_df.std()
                ax.get_yaxis().set_visible(False)
                tmpFileName = dirHistPlots+circuit+'_th_0_'+str(theta_0)+'_ph_0_'+str(phi_0)+'_'+qvf_idx+'_distribution_histogram_' + str(tmp_mean[0])+'_'+str(tmp_stddev[0])+'.pdf'
                fig.savefig(tmpFileName, bbox_inches='tight')
                plt.close()


/usr/local/lib/python3.10/site-packages/seaborn/distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.10/site-packages/seaborn/distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.10/site-packages/seaborn/distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level functi

In [ ]:

#plotting collages

dirHeatmapPlots = '../plots/double_FI/heatmaps/'
dirHistPlots = '../plots/double_FI/histograms/'
dirCollagePlots = '../plots/double_FI/collage/'

if not os.path.exists(dirHeatmapPlots):
    os.makedirs(dirHeatmapPlots)
if not os.path.exists(dirHistPlots):
    os.makedirs(dirHistPlots)
if not os.path.exists(dirCollagePlots):
    os.makedirs(dirCollagePlots)

##considering only phi angles up to pi
theta_list_tex = ['0', '', '', '$\\frac{\pi}{4}$', '', '',
                  '$\\frac{\pi}{2}$', '', '', '$\\frac{3\pi}{4}$', '', '', '$\pi$']
phi_list_tex = ['$\pi$', '', '',  '$\\frac{3\pi}{4}$', '', '',
                '$\\frac{\pi}{2}$', '', '', '$\\frac{\pi}{4}$', '', '', '0']

### plotting collage of qvf heatmaps
def my_heatmap_plot(*args, **kwargs):

    pd_series_second_phi = args[0]
    pd_series_second_theta = args[1]
    pd_series_qvf = args[2]
    pd_dataframe = pd.DataFrame({
        pd_series_second_phi.name: pd_series_second_phi,
        pd_series_second_theta.name: pd_series_second_theta,
        pd_series_qvf.name: pd_series_qvf

    })

    pd_dataframe = pd_dataframe.pivot(
        pd_series_second_phi.name, pd_series_second_theta.name, pd_series_qvf.name)
    pd_dataframe.columns.name = '$\\theta$ shift'
    pd_dataframe.index.name = '$\\phi$ shift'

    param = {'label': 'QVF'}

    divnorm = colors.TwoSlopeNorm(vmin=0, vcenter=0.5, vmax=1)
    rdgn = sns.diverging_palette(
        h_neg=130, h_pos=10, s=200, l=55, sep=20, as_cmap=True)
    sns.set(font_scale=1.3)

    ax = sns.heatmap(pd_dataframe, xticklabels=theta_list_tex,
                     yticklabels=phi_list_tex, cmap=rdgn, cbar_kws=param, vmin=0, vmax=1)



for circuit in circuits:
    # get a list of qubit columns (circuit may have different number of qubits now)
    colNames = new_qvfDF_noise[new_qvfDF_noise['circuitName'] == circuit].columns
    QVF_list = ['QVF_circuit']
    # uncomment this line to include individual qubit analysis
    QVF_list.extend([x for x in colNames if re.search('QVF_qubit_.*', x)])
    for qvf_idx in QVF_list:
        g = sns.FacetGrid(new_qvfDF_noise, row="firstPhi", col="firstTheta", margin_titles=True)
        g.map(my_heatmap_plot, "secondPhi", "secondTheta", qvf_idx)
        g.savefig(dirCollagePlots+'collage_'+circuit+'_'+qvf_idx+'.pdf')


In [10]:
#Plot 3D-scatter of QVF for the second injection with the first fixed

phi_list_tex = ['0', '$\\frac{\pi}{4}$', '$\\frac{\pi}{2}$', '$\\frac{3\pi}{4}$', '$\pi$']
phi_list_ticks = [0, np.pi/4, np.pi/2, (3*np.pi)/4, np.pi]

theta_list_tex = ['0', '$\\frac{\pi}{4}$', '$\\frac{\pi}{2}$', '$\\frac{3\pi}{4}$', '$\pi$']
theta_list_ticks = [0, np.pi/4, np.pi/2, (3*np.pi)/4, np.pi]

dir3dPlots = '../plots/double_FI/3dPlots/'



if not os.path.exists(dir3dPlots):
    os.makedirs(dir3dPlots)



for circuit in circuits:
    # get a list of qubit columns (circuit may have different number of qubits now)
    colNames = new_qvfDF_noise[new_qvfDF_noise['circuitName'] == circuit].dropna(
        axis=1).columns
    QVF_list = ['QVF_circuit']
    # uncomment this line to include individual qubit analysis
    QVF_list.extend([x for x in colNames if re.search('QVF_qubit_.*', x)])

    for qvf_idx in QVF_list:
        if qvf_idx != 'QVF_circuit':
            continue
        for phi_0 in phi_list:
            for theta_0 in theta_list:
                qvf_tmp = new_qvfDF_noise[(new_qvfDF_noise['circuitName'] == circuit) & (new_qvfDF_noise['firstPhi'] == phi_0) & (new_qvfDF_noise['firstTheta'] == theta_0)]
                fig = plt.figure(figsize=(6, 5))
            
                param = {'label': 'QVF'}

                single_QVF = qvf_tmp['QVF_circuit'][(qvf_tmp['secondPhi']==0) & (qvf_tmp['secondTheta']==0)]
                
                divnorm = colors.TwoSlopeNorm(vmin=0, vcenter=single_QVF.values[0], vmax=1)
                rdgn = sns.diverging_palette( center='light',
                    h_neg=130, h_pos=10, s=200, l=55, sep=1, as_cmap=True)
                # sns.set(font_scale=1.3)
                
                ax = plt.axes(projection='3d')
                
                ax.set_xlim3d(0, np.pi)
                ax.set_ylim3d(0, np.pi)
                ax.set_zlim3d(0, 1)

                

                
                z_single_fault = single_QVF.values[0]*np.ones((len(theta_list),len(phi_list)))

                ax.set_xticks(phi_list_ticks)
                ax.set_xticklabels(phi_list_tex)
                ax.set_xlabel('$\\phi$ shift')
                
                ax.set_yticks(theta_list_ticks)
                ax.set_yticklabels(theta_list_tex)
                ax.set_ylabel('$\\theta$ shift')

                ax.set_zlabel('QVF', rotation=90)

                ax.invert_xaxis()


                surf = ax.scatter3D(qvf_tmp['secondPhi'], qvf_tmp['secondTheta'], qvf_tmp['QVF_circuit'], 
                                    c=qvf_tmp['QVF_circuit'].values, cmap=rdgn, norm=divnorm, alpha = 1, 
                                    edgecolors='black', linewidths=0.4)
                
                x =np.outer(theta_list, np.ones(13))
                ax.plot_surface(x, x.T, z_single_fault, alpha = 0.2, 
                                rstride = 13, cstride = 13, linewidth=0)
                fig.colorbar(surf)
                
                fig.savefig(dir3dPlots+'3d'+circuit+'_th_0_'+str(theta_0)+'_ph_0_'+str(phi_0)+'_'+qvf_idx +'_heatmap.pdf', bbox_inches='tight')
                plt.close()

/tmp/ipykernel_346495/4280965333.py:76: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  fig.savefig(dir3dPlots+'3d'+circuit+'_th_0_'+str(theta_0)+'_ph_0_'+str(phi_0)+'_'+qvf_idx +'_heatmap.pdf', bbox_inches='tight')
/tmp/ipykernel_346495/4280965333.py:76: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  fig.savefig(dir3dPlots+'3d'+circuit+'_th_0_'+str(theta_0)+'_ph_0_'+str(phi_0)+'_'+qvf_idx +'_heatmap.pdf', bbox_inches='tight')
